In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import os
import scipy.io as sio

from statsmodels.tsa.ar_model import AR

import tqdm
import  time

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.simplefilter('ignore') #忽略警告

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import xgboost

In [3]:
sample_rate = 256
origin_channel = 16


SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

CLIP_FORWARD = 1 #首部裁掉时间
CLIP_BACKWARD = 1 #尾部裁掉时间

trial_time = 3 #segment second


#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频

from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
def load_data(filename):

    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix

    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate] #首部 尾部 进行裁剪
   
    return data 

In [5]:
def separate(data , label , overlap_length):
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0
    
    while idx<=data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)
        
    return np.array(train_data) , np.array(train_labels)

In [6]:
def shuffle_t_v(filenames):
    # np.random.shuffle(filenames)
    
    return np.random.choice(filenames , size=10) #20次的计算准确率中 每次随机选择10个样本进行训练测试

def combine(freq):    
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/0/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/1/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/2/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/3/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/4/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/5/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/6/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/7/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/8/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('../incremental/data/base_rf/%s/0/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('../incremental/data/base_rf/%s/1/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('../incremental/data/base_rf/%s/2/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('../incremental/data/base_rf/%s/3/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('../incremental/data/base_rf/%s/4/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('../incremental/data/base_rf/%s/5/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('../incremental/data/base_rf/%s/6/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('../incremental/data/base_rf/%s/7/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('../incremental/data/base_rf/%s/8/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    train_data = np.concatenate((train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                                 train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                                 train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                                 ))
    
    train_labels = np.concatenate((train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                                   train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                                   train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                                    ))
    
    #产生索引并置乱
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)

    #将训练数据置乱
    train_data = train_data[idx_train_data]
    train_labels = train_labels[idx_train_data]
        
    return train_data , train_labels


In [7]:
def session_data_labels(session_id , freq , is_training):
    if is_training:
        overlap_length = 256*2
    else:
        overlap_length = 0
    
    str_freq = str(freq)
    
    subjcets = os.listdir('../incremental/data/incremental/%s/s%d/' % (str_freq , session_id)) #受试者ID
    
    data = []
    labels = []
    
    for subjcet in subjcets:
        filenames = os.listdir('../incremental/data/incremental/%s/s%d/%s/' % (str_freq , session_id , subjcet))
        
        person = np.concatenate([load_data('../incremental/data/incremental/%s/s%d/%s/%s' % (str_freq , session_id , subjcet , filename)) for filename in filenames] , axis = 0)
        
        person_data , person_label = separate( person , label = int(subjcet) , overlap_length = overlap_length)
        
        data.append(person_data)
        labels.append(person_label)
    
    #合并数据
    data = np.concatenate(data)
    labels = np.concatenate(labels)
    
    #shuffle
    idx_data = list(range(data.shape[0]))
    np.random.shuffle(idx_data)

    data = data[idx_data]
    labels = labels[idx_data]
    
    return data , labels


In [8]:
def get_center(data , label):
    centers = []
    
    for label_id in range(9): #一共9个受试者
        equal_idx = np.where(label == label_id)
    
        center = np.mean(data[equal_idx] , axis = 0)
        centers.append(center)
        
    return np.array(centers)

def get_center_simple(data):
    '''
    计算单个用户的脑电的中心
    '''
    return np.mean(data , axis=0)

In [9]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input , Deconv2D , UpSampling2D

In [10]:
encoder_input = Input(shape = (256*3 , 16 , 1))
#encoder
#VGG16结构
encoder = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder_input)
encoder = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = MaxPooling2D(pool_size=(2, 2))(encoder)
encoder = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = MaxPooling2D(pool_size=(2, 2))(encoder)
encoder = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = MaxPooling2D(pool_size=(2, 2))(encoder)
#new_add
encoder_output = Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder_output = MaxPooling2D(pool_size=(2, 2))(encoder_output)
#shape = encoder_output.get_shape().as_list()

encoder_output = Flatten()(encoder_output)
encoder_output = Dense(128 , activation='relu')(encoder_output)
encoder_output = Dense(64 , activation='relu')(encoder_output)

#decoder
decoder = Dense(128 , activation='relu')(encoder_output)
decoder = Dense(48*1*32 , activation='relu')(decoder)
decoder = Reshape(target_shape=(48,1,32))(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = Deconv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = Deconv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = Deconv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder_output = Deconv2D(1, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)


In [11]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

# PCA降维

## kmeans聚类

In [11]:
for freq in [6]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  6
session :  1 [[-0.09447385  0.02152621]
 [ 0.17780232  0.02640878]
 [-0.31032544  0.00113128]
 [ 0.09829488  0.00733593]
 [ 0.3119625   0.04017555]
 [ 0.23162887 -0.04797169]
 [-0.19264477  0.00541287]
 [-0.07017577 -0.0377712 ]
 [-0.01599316  0.00518708]] 0.0753941
session :  3 [[-2.50459433e-01 -3.89048690e-03]
 [ 9.37072814e-01 -1.36329024e-03]
 [-1.38936758e-01  3.40981758e-03]
 [ 3.31222825e-02  8.69151577e-03]
 [-1.23882806e-02 -2.05367315e-03]
 [-9.23799872e-02 -9.16034542e-03]
 [-2.16429844e-01 -9.18807928e-04]
 [ 7.58526772e-02  1.04574477e-02]
 [-1.14069864e-01 -1.30800647e-03]] 0.052319400000000016
session :  5 [[-0.29962343 -0.02279103]
 [ 1.4364731  -0.00462301]
 [-0.08738236  0.01413906]
 [-0.18177633  0.00171182]
 [-0.00922005  0.01715439]
 [-0.24331377 -0.00199639]
 [-0.1369013   0.00972961]
 [-0.20821495 -0.00338747]
 [-0.26914984 -0.01489906]] 0.05688499999999941
session :  6 [[-0.14546417  0.00397042]
 [ 0.3117853  -0.04115033]
 [ 0.04934029  0.02871062]
 [

InternalError: cuDNN Backward Filter function launch failure : input shape([32,64,768,16]) filter shape([3,3,64,128])
	 [[Node: training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Conv2DBackpropFilter = Conv2DBackpropFilter[T=DT_FLOAT, _class=["loc:@training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Conv2D"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/gradients/conv2d_transpose_7/Relu_grad/ReluGrad, training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Shape, training_1/Adam/gradients/conv2d_transpose_7/conv2d_transpose_grad/Conv2DBackpropFilter-2-TransposeNHWCToNCHW-LayoutOptimizer)]]

In [11]:
for freq in [7.5]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  7.5
session :  1 [[ 0.01434443  0.0054415 ]
 [-0.01400757  0.00109773]
 [ 0.05663577 -0.00079195]
 [-0.04609253 -0.00549186]
 [-0.02569692  0.00302371]
 [ 0.07353924 -0.00426977]
 [-0.05242095 -0.00332672]
 [ 0.04022407 -0.00111593]
 [-0.00041342  0.00190035]] 0.0722994
session :  3 [[-0.00094921 -0.00457187]
 [ 0.07031219  0.00742467]
 [-0.02478894 -0.00170506]
 [ 0.01748855 -0.00453342]
 [-0.04395022  0.00873065]
 [-0.01362019  0.00231636]
 [ 0.02611357 -0.00470944]
 [ 0.00909596 -0.00490663]
 [-0.01045498 -0.00330922]] 0.06674129999999989
session :  5 [[ 3.6440637e-02 -1.3496061e-03]
 [-2.3284905e-02  2.8257251e-03]
 [ 6.5395549e-02 -9.1374066e-04]
 [-5.0541103e-02 -2.5110247e-03]
 [-1.2639939e-03  2.1623841e-03]
 [ 5.1325273e-02  9.9537858e-05]
 [-3.1367294e-02 -8.0756581e-04]
 [ 7.8378811e-02 -2.4343953e-04]
 [-2.4218241e-02 -3.0707659e-03]] 0.06572109999999931
session :  6 [[-7.9046851e-03  3.7753689e-05]
 [ 1.6243696e-01 -2.7789571e-04]
 [-2.9323896e-02  7.5860752e-04]
 

In [11]:
for freq in [8.5]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  8.5
session :  1 [[-0.01969927 -0.00614698]
 [ 0.09092501 -0.00877074]
 [ 0.01948867  0.02933999]
 [-0.00899126  0.00263531]
 [-0.01665842 -0.00424609]
 [ 0.0136526   0.02408396]
 [-0.01615458 -0.00238109]
 [-0.0110978   0.00156123]
 [-0.00530589  0.00419104]] 0.0628644
session :  3 [[ 0.01539399  0.02379128]
 [-0.01750782 -0.00271256]
 [ 0.08235874 -0.0091057 ]
 [-0.00445181 -0.00116051]
 [-0.01514146  0.00073359]
 [ 0.02187754  0.02796924]
 [-0.02132517 -0.00565634]
 [-0.00848832 -0.00197002]
 [-0.01788202 -0.00569959]] 0.05546169999999995
session :  5 [[-0.01281557 -0.002678  ]
 [ 0.08550283 -0.00271057]
 [-0.00500562  0.00371323]
 [ 0.00270334  0.01175892]
 [-0.01110264  0.00199661]
 [-0.01688745 -0.00501842]
 [-0.01151746  0.00012789]
 [-0.00923877  0.00320081]
 [-0.01519627 -0.00381025]] 0.05574060000000003
session :  6 [[-0.02241479 -0.00247682]
 [ 0.02571617 -0.00767023]
 [ 0.07060371 -0.00834391]
 [ 0.01694128  0.02998658]
 [-0.01545161  0.01452984]
 [-0.01952418 -0.00

In [11]:
for freq in [10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  10
session :  1 [[-3.5306129e-03 -6.4287230e-04]
 [ 1.8940449e-02 -7.2372693e-04]
 [ 3.8420421e-03  1.7443667e-03]
 [-3.9269132e-04  5.6607096e-04]
 [-2.2643523e-03  5.8840972e-04]
 [-4.1958014e-03 -5.0490384e-04]
 [ 8.2645757e-04  2.6698883e-03]
 [-3.2479565e-03 -3.1466356e-05]
 [-4.7385069e-03 -5.9664028e-04]] 0.0599142
session :  3 [[-7.3933206e-03 -4.8460756e-04]
 [ 3.8379781e-02 -1.2296221e-03]
 [ 6.2044505e-03  3.0324527e-03]
 [-5.6392956e-04  1.0254397e-03]
 [ 3.5400398e-02  1.0468729e-05]
 [ 4.0754292e-02 -2.2830728e-03]
 [ 7.1406690e-03  3.2751812e-03]
 [ 1.1552724e-03  1.3596849e-03]
 [-7.8349365e-03 -6.3385128e-04]] 0.052649600000000074
session :  5 [[-2.6629707e-03 -1.3150883e-04]
 [ 3.2910853e-04  1.6367510e-05]
 [ 6.9533591e-03 -4.6195151e-04]
 [ 2.3899609e-03  5.8901962e-04]
 [-1.7301593e-03  2.6212740e-05]
 [ 1.6405578e-03  2.7575187e-04]
 [-2.3593791e-03 -2.4875448e-04]
 [ 1.1516995e-03 -3.7068698e-05]
 [-2.5512588e-03  2.8122964e-05]] 0.05377960000000037
sessi

## DBSCAN聚类

In [ ]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        dbscan = DBSCAN()
        start = time.clock()
        res = dbscan.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)

In [ ]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9 , tol=0.001)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

## GMM聚类

In [12]:
for freq in [6]:    
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)

freq :  6
session :  1 [[-0.00757415  0.00119063]
 [ 0.03676995 -0.00802503]
 [ 0.01329238  0.00966795]
 [-0.01064601 -0.00112506]
 [-0.01651772 -0.00355853]
 [-0.00544219 -0.00469479]
 [ 0.00576645  0.00857607]
 [-0.00307717  0.00639659]
 [ 0.00080191 -0.00081612]] 0.0324387
session :  3 [[           nan            nan]
 [ 9.4452828e-02  7.6971373e-05]
 [-1.1806602e-02 -9.6239910e-06]
 [           nan            nan]
 [           nan            nan]
 [           nan            nan]
 [           nan            nan]
 [           nan            nan]
 [           nan            nan]] 0.03578440000000027
session :  5 [[        nan         nan]
 [ 0.1563898  -0.00109791]
 [-0.02786399 -0.00393227]
 [ 0.00025336  0.00585341]
 [-0.02421495 -0.00200753]
 [-0.01284182  0.00470128]
 [-0.01878054 -0.00112652]
 [-0.00496752  0.00649125]
 [-0.01629715  0.00075972]] 0.04120349999999995
session :  6 [[-0.03168932  0.00158185]
 [ 0.09043541  0.02288529]
 [ 0.01700126 -0.01470964]
 [ 0.08108009 -0.0209

In [12]:
for freq in [7.5]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)

freq :  7.5
session :  1 [[ 9.4154358e+00 -6.4416417e-06]
 [-3.3637600e+00  7.8739570e-03]
 [ 2.3773577e+00  4.3822862e-02]
 [-7.2009974e+00 -5.4437004e-02]
 [ 8.7993031e+00 -9.1961935e-02]
 [-1.5915577e+00  3.1003214e-02]
 [ 4.6752100e+00  4.6495315e-02]
 [-2.4704182e+00  6.5240353e-02]
 [-8.5239449e+00 -1.2966301e-02]] 0.0678126
session :  3 [[-3.7276573e+00  1.2103634e-02]
 [ 5.6900883e+00  2.0343222e-02]
 [ 2.3716626e+00  1.6483216e-01]
 [-7.0868492e+00 -9.7047232e-02]
 [-2.3997145e+00  4.9333512e-03]
 [ 8.3823442e+00 -1.0304502e-01]
 [ 3.3365636e+00  2.3169173e-02]
 [-7.3241287e-01  7.5617820e-02]
 [ 4.8878803e+00 -1.1093919e-03]] 0.03427680000000022
session :  5 [[-1.2619125e+00  1.9357963e-02]
 [ 1.1858756e+01 -1.7231476e-02]
 [-6.6963682e+00  1.5455200e-02]
 [ 1.2719468e+01  8.5513785e-02]
 [-4.6376929e+00 -4.1973196e-02]
 [ 5.7854503e-01 -6.2906273e-02]
 [ 1.3995359e+01  2.8194884e-02]
 [-7.4599881e+00  5.5054501e-02]
 [-3.4662671e+00  9.7579183e-03]] 0.04269630000000024
sessi

In [12]:
for freq in [8.5]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)

freq :  8.5
session :  1 [[-1.98812522e-02 -2.13570055e-03]
 [ 1.17314726e-01 -2.18753261e-03]
 [ 1.00285804e-03  1.18445614e-02]
 [-3.91838886e-03  4.48140818e-05]
 [            nan             nan]
 [-1.39220003e-02  7.54498785e-07]
 [ 5.57956286e-03  1.36007546e-02]
 [-1.28340768e-03  1.10963825e-03]
 [-8.11736844e-03  5.38166729e-04]] 0.0280181
session :  3 [[-0.01709497 -0.00144547]
 [ 0.10144152 -0.00250478]
 [ 0.00312068  0.01303584]
 [-0.00561121 -0.00035695]
 [-0.01057623 -0.00142007]
 [-0.02005044 -0.0029348 ]
 [ 0.00780104  0.01515075]
 [        nan         nan]
 [        nan         nan]] 0.0500642
session :  5 [[-0.01795324 -0.00192196]
 [ 0.10405303 -0.0025102 ]
 [ 0.01032302  0.00750644]
 [        nan         nan]
 [-0.02146114 -0.0044777 ]
 [-0.01604699  0.00062339]
 [-0.00510438  0.00478348]
 [        nan         nan]
 [        nan         nan]] 0.040785699999999814
session :  6 [[-0.01799958 -0.00050568]
 [ 0.07684328 -0.00165288]
 [ 0.02034799 -0.00760513]
 [ 0.01032

In [12]:
for freq in [10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  10
session :  1 [[-7.04392076e-01 -2.09315475e-02]
 [ 5.63012457e+00 -5.56501793e-03]
 [-7.09827363e-01  8.71649683e-02]
 [ 5.40426874e+00 -8.65458220e-04]
 [ 5.84280252e+00  6.59412146e-03]
 [ 5.74705696e+00 -1.39561035e-02]
 [-7.06118286e-01  2.16792021e-02]
 [ 5.56143188e+00  9.74743068e-03]
 [ 5.70149660e+00 -7.20915094e-04]] 0.0369429
session :  3 [[-2.3196855e+00 -2.0137161e-02]
 [ 1.8956675e+01 -1.7865689e-02]
 [ 1.5790914e+01 -3.0058125e-02]
 [ 1.7480545e+01  4.4512719e-02]
 [ 2.0140879e+01  3.8847495e-03]
 [-2.2855928e+00  1.4201346e-01]
 [ 1.7986347e+01  5.6252726e-03]
 [ 1.8537596e+01  1.5331234e+00]
 [ 1.7326328e+01 -7.7227056e-01]] 0.018053500000000167
session :  5 [[-0.07920576  0.00414107]
 [ 0.03626357 -0.01885132]
 [ 0.08077472  0.00035716]
 [ 0.13292567  0.03924015]
 [ 0.01443447 -0.01659924]
 [ 0.11265236  0.02386106]
 [ 0.08547084  0.00374885]
 [ 0.06262187 -0.01107494]
 [ 0.104385    0.01770191]] 0.019675999999999583
session :  6 [[ 0.15824093 -0.00791765]


# LDA降维
## kmeans聚类

In [12]:
for freq in [6]:    
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)

freq :  6
session :  1 [[  6.75123609  -4.57586092]
 [-33.84670175  -2.83502074]
 [  3.05914127   1.98361034]
 [ -5.3960048    6.32186846]
 [  5.30013813   7.51237301]
 [ 10.6597653   -6.67237999]
 [ 10.14748369  -9.7782502 ]
 [  5.04905787   3.62668365]
 [-36.93628044  -4.32914698]] 0.0716697
session :  3 [[-28.67896288  36.09287577]
 [242.57945232   2.06855949]
 [-26.73482791 -13.72683933]
 [-33.61067494  -6.9948853 ]
 [-42.14849265  29.85758438]
 [-25.43328778 -27.87712034]
 [-23.42748812  -6.95079989]
 [-33.78237814   1.26560509]
 [238.20547331   2.05100645]] 0.050294399999999406
session :  5 [[-1.15261131e+02 -6.98054782e+00]
 [ 9.18371305e+02 -1.15276787e-01]
 [-1.11918780e+02  1.43760576e+01]
 [-1.09583148e+02  2.69493067e+01]
 [-1.14487946e+02 -1.19338307e+01]
 [-1.16610625e+02  4.35957259e+00]
 [-1.21094239e+02 -4.92224251e+00]
 [-1.15592603e+02 -9.72183666e+00]
 [ 9.21196383e+02 -3.14294997e-01]] 0.049805700000000286
session :  6 [[ 84.22514953   3.42597544]
 [-28.64020185  -

In [12]:
for freq in [7.5]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  7.5
session :  1 [[ 10.48708886   0.59894977]
 [ -6.17721894   3.06670814]
 [ -8.50943705 -11.91227707]
 [-14.2407277   10.99919096]
 [  0.42564166  -2.92603461]
 [ 19.39662059   4.13285668]
 [  5.26363825  -2.70008124]
 [ -4.25868508  -2.84719218]
 [ -4.16653915   1.50279635]] 0.0678909
session :  3 [[-4.23270758e+00 -1.89857610e+01]
 [-9.94676403e+01  1.08951898e+01]
 [ 1.68226486e+01  1.83356020e+01]
 [ 1.16671201e+01 -7.16689115e+00]
 [ 2.86190067e+01  4.22944539e+01]
 [ 8.47285006e-03 -2.08748533e+01]
 [ 5.22129902e+00 -2.21734086e+01]
 [ 1.07332518e+01 -1.35036439e+01]
 [ 1.41275583e+01 -3.46623874e+00]] 0.0548801000000001
session :  5 [[-160.84425315    7.33351907]
 [  17.81970958   11.71742497]
 [  12.65046671  -57.91774285]
 [  14.08135838  -21.74802619]
 [  42.09340313   39.88334999]
 [  18.26177826   23.27692247]
 [  21.79665454  -10.20305904]
 [  19.18397491    3.20013948]
 [  15.01729432    4.4395102 ]] 0.050948899999998964
session :  6 [[  0.97842412 -10.56364079]

In [12]:
for freq in [8.5]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  8.5
session :  1 [[ -68.24200524   34.03683997]
 [ 794.16629018    5.87131975]
 [ -81.02400044 -251.05079179]
 [-110.86975652   43.28758979]
 [ -89.23688227   30.84102552]
 [-124.41881614   30.48453099]
 [ -96.20297214   32.71007102]
 [-118.14464393   29.96320052]
 [-106.09639268   43.84809103]] 0.055306999999999995
session :  3 [[-117.75503025  -20.39284986]
 [ 871.35450684   14.96775953]
 [ -40.23848618 -200.04140767]
 [-115.86275984   32.5385392 ]
 [ -94.47647934   66.47965018]
 [-125.61315874    3.39976957]
 [-131.71870454   27.38089398]
 [-135.0659719    42.46403627]
 [-110.58567632   33.20846332]] 0.049395700000000264
session :  5 [[-177.97167563   35.78124995]
 [1185.8927611    -3.79185728]
 [-116.09275496  -28.23742627]
 [-107.31060664  135.43128356]
 [-164.07165618  -40.12007726]
 [-155.98695038  -11.17108207]
 [-143.35857891  -23.70459745]
 [-163.29297842  -48.63103559]
 [-157.90484446  -15.79079063]] 0.04749730000000074
session :  6 [[  74.8610669   -37.26251122]
 [-

In [12]:
for freq in [10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

freq :  10
session :  1 [[  99.06889702 -128.16995498]
 [ -36.23004268   17.16267253]
 [ 204.10625904   77.59037287]
 [  -9.43948641  -12.72038214]
 [ -55.80465201    0.75829626]
 [ -54.40976224   12.16705013]
 [ -39.5473481    -8.23853679]
 [ -58.327276     23.61684089]
 [ -49.41658863   17.83364122]] 0.062126799999999996
session :  3 [[ 356.88567008  162.49349854]
 [-111.83373168  -10.75258167]
 [ 203.48710182 -172.14706175]
 [  35.25055516 -161.81249551]
 [-100.26511676   44.62640108]
 [ -74.65688295   16.80254605]
 [ -88.34091999   29.3948772 ]
 [-115.41665786   51.25212437]
 [-105.11001782   40.14269169]] 0.05073250000000007
session :  5 [[-139.26019048   89.276422  ]
 [ 932.11601878   22.17163165]
 [-118.00255868 -115.90829866]
 [   9.06681296 -158.99598958]
 [-139.327617      8.3977571 ]
 [-145.12668673  -95.21954984]
 [-141.96193331   70.96679562]
 [-125.03212451   95.38450996]
 [-132.47172103   83.92672174]] 0.048028200000000076
session :  6 [[-36.43058268  -4.72824337]
 [186.

## DBSCAN聚类

In [ ]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        dbscan = DBSCAN()
        start = time.clock()
        res = dbscan.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

In [ ]:
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9 , tol=0.001)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
                
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    

## GMM聚类

In [12]:
for freq in [6]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  6
session :  1 [[  0.52808416  -1.97458061]
 [ 63.96650273   2.52993535]
 [-11.30750346  24.63174969]
 [-10.44471334  -5.22354625]
 [ -6.09485373 -18.49326385]
 [-10.16793729   6.37894617]
 [ -7.00462278   1.52169142]
 [-12.59602584   5.36125008]
 [ -6.37378464 -14.52361798]] 0.1221087
session :  3 [[-3.27848917e+02 -3.89486835e+01]
 [ 2.65637787e+03  2.37976275e-01]
 [-3.31154615e+02  1.80642991e+01]
 [-3.29451468e+02 -4.38290894e+00]
 [-3.45975926e+02  3.79454141e+01]
 [-3.30842217e+02  6.04515121e+00]
 [-3.31526787e+02 -3.29425611e+01]
 [-3.31219661e+02  2.12076038e+01]
 [-3.28273551e+02 -8.64994607e+00]] 0.0410096000000002
session :  5 [[-2.26833527e+02  5.60229965e-01]
 [ 1.81017217e+03 -2.72512221e-02]
 [-2.26129943e+02 -2.14050855e+01]
 [-2.24441289e+02  1.08830998e+01]
 [-2.27785734e+02 -7.07339358e+00]
 [-2.25419208e+02  1.29765544e+01]
 [-2.26354633e+02  6.76496371e+00]
 [-2.26470305e+02 -4.14643543e+00]
 [-2.24612060e+02  1.52897234e+01]] 0.05775349999999957
session 

In [12]:
for freq in [7.5]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32a , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  7.5
session :  1 [[-16.12204552  13.197131  ]
 [ 49.13704646  -7.14565998]
 [-13.89309873 -19.89130643]
 [  9.39043019  12.56562191]
 [-30.02708329   6.38404183]
 [ -9.90514996  -1.46626863]
 [  2.93933385  10.10119597]
 [ -6.77235547 -23.91562158]
 [ 15.29571439  10.15338593]] 0.0274266
session :  3 [[  5.05143177   9.88070623]
 [-30.31461142 -39.04641178]
 [ 51.3347853  -26.4545702 ]
 [ -5.59011496 -15.62269788]
 [-21.01271949  18.18415968]
 [ 13.5357872   22.27656031]
 [-16.28022791  -4.56987999]
 [ -6.65846031  18.56271861]
 [  9.96976996  16.83984354]] 0.023601200000000322
session :  5 [[ 19.54022758 -40.12651211]
 [-92.66774924  -2.73434075]
 [ 27.13840697   6.29074855]
 [  9.66915892  64.84459101]
 [  9.70970178  -2.6398861 ]
 [ -8.36179049  -3.64268381]
 [ 14.24516883 -13.30696831]
 [ 10.85513734   3.50938225]
 [  9.87173832 -12.19433074]] 0.022774099999999464
session :  6 [[  35.69229109    4.39248566]
 [-106.22312339   18.02645421]
 [ -15.18269548  -14.76667947]
 [  1

In [13]:
for freq in [8.5]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  8.5
session :  1 [[ 3.85035862e+03  1.88817329e-01]
 [-4.78946497e+02  3.17428329e+00]
 [-4.82875541e+02 -5.86429349e+01]
 [-4.82031756e+02  4.50372048e+01]
 [-4.87303013e+02  1.52211838e+01]
 [-4.78189726e+02 -1.67178691e+01]
 [-4.81199923e+02 -2.87086438e+00]
 [-4.81264859e+02  1.76521826e+01]
 [ 3.85461515e+03 -8.58010767e-02]] 0.030942
session :  3 [[-4.34275969e+02 -3.63409712e+01]
 [ 3.46129145e+03 -2.13153698e-01]
 [-4.32217598e+02  3.03356401e+01]
 [-4.39526699e+02 -1.05743910e+02]
 [-4.27953863e+02  3.79054063e+00]
 [-4.35555222e+02  4.31739263e+01]
 [-4.32362057e+02  1.29659791e+01]
 [-4.30301055e+02  4.17930470e+01]
 [-4.29098987e+02  1.02389020e+01]] 0.022821200000000097
session :  5 [[-4.20998980e+02  1.71281668e+01]
 [ 3.38384102e+03  1.13375178e+00]
 [-4.40783302e+02  1.91263217e+02]
 [-4.26673646e+02 -4.20564457e+01]
 [-4.10297313e+02 -6.57837099e+01]
 [-4.21229146e+02 -2.54161484e+01]
 [-4.14354303e+02 -3.51422878e+01]
 [-4.24597877e+02  8.31755163e+00]
 [-4.24

In [13]:
for freq in [10]:
    print('freq : ' , freq)
    
    meta_data , meta_labels = combine(freq)
    meta_data = np.expand_dims(meta_data , axis=-1)
    meta_data = (meta_data-np.mean(meta_data)) / np.std(meta_data)
    
    #自编码器的训练
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(meta_data , meta_data , batch_size=32 , epochs=50 , verbose=0)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.expand_dims(data , axis=-1)
        data = (data-np.mean(data)) / np.std(data)
        
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_pca = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        print('session : ' , session_id , _centers , time1 + time2)
    

freq :  10
session :  1 [[ 75.70888615  42.59630989]
 [-11.96229248  -3.00689028]
 [ 54.60406156 -52.3771864 ]
 [-41.46671393   8.65363237]
 [-16.67423087   8.63803796]
 [ -2.5959219    3.00453815]
 [-18.57665284  -2.94336743]
 [-23.05189408   2.5357249 ]
 [-16.00388487  -7.07088957]] 0.030183599999999998
session :  3 [[138.57764808  26.9338484 ]
 [-43.97324353 -34.28919347]
 [-62.3761722   46.29307612]
 [ 46.20759894 -72.75993648]
 [190.65198244  20.18648013]
 [-84.660329    78.25037347]
 [-68.28977403  -5.79583704]
 [-59.63328008 -21.15809819]
 [-56.50443061 -37.66071295]] 0.023428100000000285
session :  5 [[-291.51570225  -24.5141511 ]
 [2232.49878464    4.44521943]
 [-232.07400594 -215.10783831]
 [-290.26761119   49.0997707 ]
 [-285.12407199   22.12159398]
 [-282.9466175    63.53015256]
 [-293.89334693   33.65807337]
 [-277.67179554   49.45047836]
 [-278.96740996   17.21467437]] 0.02276279999999975
session :  6 [[-43.76018227  -2.48238482]
 [210.48568125 -15.07705319]
 [ 38.3953240